# Wave analysis: finding typical deepwater wave condition

## Reading data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import datetime 

In [2]:
table = pd.read_csv('wavedata.txt', delim_whitespace=True)

In [3]:
table['datetime'] = table.apply(lambda row: 
                                datetime.datetime(int(row.year), int(row.month), int(row.day), int(row.hour)), 
                                axis=1)

In [4]:
tableJan = table[table['month']==1]
tableFeb = table[table['month']==2]
tableMar = table[table['month']==3]
tableApr = table[table['month']==4]
tableMay = table[table['month']==5]
tableJun = table[table['month']==6]
tableJul = table[table['month']==7]
tableAug = table[table['month']==8]
tableSep = table[table['month']==9]
tableOct = table[table['month']==10]
tableNov = table[table['month']==11]
tableDec = table[table['month']==12]

In [12]:
tableJan.head()

,year,month,day,hour,Hs,Tp,Dir,datetime
0,1989,1,1,0,2.59,9.32,41.65,1989-01-01 00:00:00
1,1989,1,1,3,2.73,9.03,40.41,1989-01-01 03:00:00
2,1989,1,1,6,2.78,9.06,40.27,1989-01-01 06:00:00
3,1989,1,1,9,2.91,9.04,40.55,1989-01-01 09:00:00
4,1989,1,1,12,3.01,9.05,41.56,1989-01-01 12:00:00


## Calculating typical wave conditions

The typical wave characteristics are calculated using Goda's method (2010) for each month:

$$ H_s = \sqrt{\frac{\Sigma H_i^2 T_i}{\Sigma T_i}} $$
$$ T = \frac{\Sigma T_i}{N} $$
$$ \theta = \frac{\Sigma \theta H_i^2 T_i}{\Sigma H_i^2 T_i} $$

where &Sigma; ranges from 1 to N; i is the sample index, N is the number of samples.

In [5]:
# checking wave direction grouping
def inGroup1(x):
    return x < 101 or x > 349

def inGroup2(x):
    return 101 <= x <= 210

In [6]:
# partitioning January wave data into groups
tableJan1 = tableJan[tableJan['Dir'].map(inGroup1)]
tableJan2 = tableJan[tableJan['Dir'].map(inGroup2)]

len(tableJan), len(tableJan1)

(7688, 7667)

For January, the second group is negligible, thus we perform analysis only for Group (subset) 1.

In [7]:
tableJan1['H2T'] = tableJan1.apply(lambda row: row['Hs']**2 * row['Tp'], axis=1)
tableJan1['thH2T'] = tableJan1.apply(lambda row: row['Dir'] * row['Hs']**2 * row['Tp'], axis=1)

/Volumes/W10/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Volumes/W10/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
# checking new columns
tableJan1.head()

,year,month,day,hour,Hs,Tp,Dir,datetime,H2T,thH2T
0,1989,1,1,0,2.59,9.32,41.65,1989-01-01 00:00:00,62.519492,2603.936842
1,1989,1,1,3,2.73,9.03,40.41,1989-01-01 03:00:00,67.299687,2719.580352
2,1989,1,1,6,2.78,9.06,40.27,1989-01-01 06:00:00,70.019304,2819.677372
3,1989,1,1,9,2.91,9.04,40.55,1989-01-01 09:00:00,76.551624,3104.168353
4,1989,1,1,12,3.01,9.05,41.56,1989-01-01 12:00:00,81.993905,3407.666692


In [9]:
# calculating the three characteristics of this January wave subset
import math
Hrep = math.sqrt(sum(tableJan1['H2T'])/ sum(tableJan1['Tp']))
Trep = tableJan1.Tp.mean()
thetarep = sum(tableJan1['thH2T'])/ sum(tableJan1['H2T'])
print(Hrep, Trep, thetarep)

2.421760019922373 8.462024259814802 51.74237495815952


Perform similar calculations for other subsets.

(TODO: Create a dictionary and iterate over the months instead of repeating this code segment.)

In [10]:
import warnings
warnings.filterwarnings('ignore')

In [11]:
tableJan1 = tableJan[tableJan['Dir'].map(inGroup1)]
tableJan2 = tableJan[tableJan['Dir'].map(inGroup2)]
print( 'Length of: subset1 ({}) and subset 2 ({})'.format(len(tableJan1), len(tableJan2)) )
Pct1 = len(tableJan1) / len(tableJan) * 100
Pct2 = len(tableJan2) / len(tableJan) * 100
tableJan1['H2T'] = tableJan1.apply(lambda row: row['Hs']**2 * row['Tp'], axis=1)
tableJan2['H2T'] = tableJan2.apply(lambda row: row['Hs']**2 * row['Tp'], axis=1)
tableJan1['thH2T'] = tableJan1.apply(lambda row: row['Dir'] * row['Hs']**2 * row['Tp'], axis=1)
tableJan2['thH2T'] = tableJan2.apply(lambda row: row['Dir'] * row['Hs']**2 * row['Tp'], axis=1)
Hrep1 = math.sqrt(sum(tableJan1['H2T'])/ sum(tableJan1['Tp']))
Hrep2 = math.sqrt(sum(tableJan2['H2T'])/ sum(tableJan2['Tp']))
Trep1 = tableJan1.Tp.mean()
Trep2 = tableJan2.Tp.mean()
thetarep1 = sum(tableJan1['thH2T'])/ sum(tableJan1['H2T'])
thetarep2 = sum(tableJan2['thH2T'])/ sum(tableJan2['H2T'])
print( '|  Jan  |  {:.2f} |  {:.2f} |   {:.1f}   | {:.1f} |'.format(Hrep1, Trep1, thetarep1, Pct1) )
print( '|  Jan  |  {:.2f} |  {:.2f} |   {:.1f}   | {:.1f} |'.format(Hrep2, Trep2, thetarep2, Pct2) )

Length of: subset1 (7667) and subset 2 (13)
|  Jan  |  2.42 |  8.46 |   51.7   | 99.7 |
|  Jan  |  1.39 |  4.61 |   120.8   | 0.2 |


In [12]:
tableFeb1 = tableFeb[tableFeb['Dir'].map(inGroup1)]
tableFeb2 = tableFeb[tableFeb['Dir'].map(inGroup2)]
print( 'Length of: subset1 ({}) and subset 2 ({})'.format(len(tableFeb1), len(tableFeb2)) )
Pct1 = len(tableFeb1) / len(tableFeb) * 100
Pct2 = len(tableFeb2) / len(tableFeb) * 100
tableFeb1['H2T'] = tableFeb1.apply(lambda row: row['Hs']**2 * row['Tp'], axis=1)
tableFeb2['H2T'] = tableFeb2.apply(lambda row: row['Hs']**2 * row['Tp'], axis=1)
tableFeb1['thH2T'] = tableFeb1.apply(lambda row: row['Dir'] * row['Hs']**2 * row['Tp'], axis=1)
tableFeb2['thH2T'] = tableFeb2.apply(lambda row: row['Dir'] * row['Hs']**2 * row['Tp'], axis=1)
Hrep1 = math.sqrt(sum(tableFeb1['H2T'])/ sum(tableFeb1['Tp']))
Hrep2 = math.sqrt(sum(tableFeb2['H2T'])/ sum(tableFeb2['Tp']))
Trep1 = tableFeb1.Tp.mean()
Trep2 = tableFeb2.Tp.mean()
thetarep1 = sum(tableFeb1['thH2T'])/ sum(tableFeb1['H2T'])
thetarep2 = sum(tableFeb2['thH2T'])/ sum(tableFeb2['H2T'])
print( '|  Feb  |  {:.2f} |  {:.2f} |   {:.1f}   | {:.1f} |'.format(Hrep1, Trep1, thetarep1, Pct1) )
print( '|  Feb  |  {:.2f} |  {:.2f} |   {:.1f}   | {:.1f} |'.format(Hrep2, Trep2, thetarep2, Pct2) )

Length of: subset1 (6607) and subset 2 (374)
|  Feb  |  2.04 |  8.14 |   52.2   | 94.4 |
|  Feb  |  0.66 |  4.58 |   122.9   | 5.3 |


In [13]:
tableMar1 = tableMar[tableMar['Dir'].map(inGroup1)]
tableMar2 = tableMar[tableMar['Dir'].map(inGroup2)]
print( 'Length of: subset 1 ({}) and subset 2 ({})'.format(len(tableMar1), len(tableMar2)) )
Pct1 = len(tableMar1) / len(tableMar) * 100
Pct2 = len(tableMar2) / len(tableMar) * 100
tableMar1['H2T'] = tableMar1.apply(lambda row: row['Hs']**2 * row['Tp'], axis=1)
tableMar2['H2T'] = tableMar2.apply(lambda row: row['Hs']**2 * row['Tp'], axis=1)
tableMar1['thH2T'] = tableMar1.apply(lambda row: row['Dir'] * row['Hs']**2 * row['Tp'], axis=1)
tableMar2['thH2T'] = tableMar2.apply(lambda row: row['Dir'] * row['Hs']**2 * row['Tp'], axis=1)
Hrep1 = math.sqrt(sum(tableMar1['H2T'])/ sum(tableMar1['Tp']))
Hrep2 = math.sqrt(sum(tableMar2['H2T'])/ sum(tableMar2['Tp']))
Trep1 = tableMar1.Tp.mean()
Trep2 = tableMar2.Tp.mean()
thetarep1 = sum(tableMar1['thH2T'])/ sum(tableMar1['H2T'])
thetarep2 = sum(tableMar2['thH2T'])/ sum(tableMar2['H2T'])
print( '|  Mar  |  {:.2f} |  {:.2f} |   {:.1f}   | {:.1f} |'.format(Hrep1, Trep1, thetarep1, Pct1) )
print( '|  Mar  |  {:.2f} |  {:.2f} |   {:.1f}   | {:.1f} |'.format(Hrep2, Trep2, thetarep2, Pct2) )

Length of: subset 1 (6714) and subset 2 (947)
|  Mar  |  1.66 |  7.74 |   51.7   | 87.3 |
|  Mar  |  0.75 |  4.19 |   141.8   | 12.3 |


In [14]:
tableApr1 = tableApr[tableApr['Dir'].map(inGroup1)]
tableApr2 = tableApr[tableApr['Dir'].map(inGroup2)]
print( 'Length of: subset 1 ({}) and subset 2 ({})'.format(len(tableApr1), len(tableApr2)) )
Pct1 = len(tableApr1) / len(tableApr) * 100
Pct2 = len(tableApr2) / len(tableApr) * 100
tableApr1['H2T'] = tableApr1.apply(lambda row: row['Hs']**2 * row['Tp'], axis=1)
tableApr2['H2T'] = tableApr2.apply(lambda row: row['Hs']**2 * row['Tp'], axis=1)
tableApr1['thH2T'] = tableApr1.apply(lambda row: row['Dir'] * row['Hs']**2 * row['Tp'], axis=1)
tableApr2['thH2T'] = tableApr2.apply(lambda row: row['Dir'] * row['Hs']**2 * row['Tp'], axis=1)
Hrep1 = math.sqrt(sum(tableApr1['H2T'])/ sum(tableApr1['Tp']))
Hrep2 = math.sqrt(sum(tableApr2['H2T'])/ sum(tableApr2['Tp']))
Trep1 = tableApr1.Tp.mean()
Trep2 = tableApr2.Tp.mean()
thetarep1 = sum(tableApr1['thH2T'])/ sum(tableApr1['H2T'])
thetarep2 = sum(tableApr2['thH2T'])/ sum(tableApr2['H2T'])
print( '|  Apr  |  {:.2f} |  {:.2f} |   {:.1f}   | {:.1f} |'.format(Hrep1, Trep1, thetarep1, Pct1) )
print( '|  Apr  |  {:.2f} |  {:.2f} |   {:.1f}   | {:.1f} |'.format(Hrep2, Trep2, thetarep2, Pct2) )

Length of: subset 1 (4953) and subset 2 (2437)
|  Apr  |  1.18 |  7.38 |   52.5   | 66.6 |
|  Apr  |  0.85 |  4.21 |   164.3   | 32.8 |


In [15]:
tableMay1 = tableMay[tableMay['Dir'].map(inGroup1)]
tableMay2 = tableMay[tableMay['Dir'].map(inGroup2)]
print( 'Length of: subset 1 ({}) and subset 2 ({})'.format(len(tableMay1), len(tableMay2)) )
Pct1 = len(tableMay1) / len(tableMay) * 100
Pct2 = len(tableMay2) / len(tableMay) * 100
tableMay1['H2T'] = tableMay1.apply(lambda row: row['Hs']**2 * row['Tp'], axis=1)
tableMay2['H2T'] = tableMay2.apply(lambda row: row['Hs']**2 * row['Tp'], axis=1)
tableMay1['thH2T'] = tableMay1.apply(lambda row: row['Dir'] * row['Hs']**2 * row['Tp'], axis=1)
tableMay2['thH2T'] = tableMay2.apply(lambda row: row['Dir'] * row['Hs']**2 * row['Tp'], axis=1)
Hrep1 = math.sqrt(sum(tableMay1['H2T'])/ sum(tableMay1['Tp']))
Hrep2 = math.sqrt(sum(tableMay2['H2T'])/ sum(tableMay2['Tp']))
Trep1 = tableMay1.Tp.mean()
Trep2 = tableMay2.Tp.mean()
thetarep1 = sum(tableMay1['thH2T'])/ sum(tableMay1['H2T'])
thetarep2 = sum(tableMay2['thH2T'])/ sum(tableMay2['H2T'])
print( '|  May  |  {:.2f} |  {:.2f} |   {:.1f}   | {:.1f} |'.format(Hrep1, Trep1, thetarep1, Pct1) )
print( '|  May  |  {:.2f} |  {:.2f} |   {:.1f}   | {:.1f} |'.format(Hrep2, Trep2, thetarep2, Pct2) )

Length of: subset 1 (2972) and subset 2 (4581)
|  May  |  1.02 |  7.16 |   53.5   | 38.7 |
|  May  |  0.92 |  4.80 |   174.2   | 59.6 |


In [16]:
tableJun1 = tableJun[tableJun['Dir'].map(inGroup1)]
tableJun2 = tableJun[tableJun['Dir'].map(inGroup2)]
print( 'Length of: subset 1 ({}) and subset 2 ({})'.format(len(tableJun1), len(tableJun2)) )
Pct1 = len(tableJun1) / len(tableJun) * 100
Pct2 = len(tableJun2) / len(tableJun) * 100
tableJun1['H2T'] = tableJun1.apply(lambda row: row['Hs']**2 * row['Tp'], axis=1)
tableJun2['H2T'] = tableJun2.apply(lambda row: row['Hs']**2 * row['Tp'], axis=1)
tableJun1['thH2T'] = tableJun1.apply(lambda row: row['Dir'] * row['Hs']**2 * row['Tp'], axis=1)
tableJun2['thH2T'] = tableJun2.apply(lambda row: row['Dir'] * row['Hs']**2 * row['Tp'], axis=1)
Hrep1 = math.sqrt(sum(tableJun1['H2T'])/ sum(tableJun1['Tp']))
Hrep2 = math.sqrt(sum(tableJun2['H2T'])/ sum(tableJun2['Tp']))
Trep1 = tableJun1.Tp.mean()
Trep2 = tableJun2.Tp.mean()
thetarep1 = sum(tableJun1['thH2T'])/ sum(tableJun1['H2T'])
thetarep2 = sum(tableJun2['thH2T'])/ sum(tableJun2['H2T'])
print( '|  Jun  |  {:.2f} |  {:.2f} |   {:.1f}   | {:.1f} |'.format(Hrep1, Trep1, thetarep1, Pct1) )
print( '|  Jun  |  {:.2f} |  {:.2f} |   {:.1f}   | {:.1f} |'.format(Hrep2, Trep2, thetarep2, Pct2) )

Length of: subset 1 (681) and subset 2 (6107)
|  Jun  |  0.84 |  6.69 |   56.1   | 9.5 |
|  Jun  |  1.04 |  5.64 |   179.5   | 84.8 |


In [17]:
tableJul1 = tableJul[tableJul['Dir'].map(inGroup1)]
tableJul2 = tableJul[tableJul['Dir'].map(inGroup2)]
print( 'Length of: subset 1 ({}) and subset 2 ({})'.format(len(tableJul1), len(tableJul2)) )
Pct1 = len(tableJul1) / len(tableJul) * 100
Pct2 = len(tableJul2) / len(tableJul) * 100
tableJul1['H2T'] = tableJul1.apply(lambda row: row['Hs']**2 * row['Tp'], axis=1)
tableJul2['H2T'] = tableJul2.apply(lambda row: row['Hs']**2 * row['Tp'], axis=1)
tableJul1['thH2T'] = tableJul1.apply(lambda row: row['Dir'] * row['Hs']**2 * row['Tp'], axis=1)
tableJul2['thH2T'] = tableJul2.apply(lambda row: row['Dir'] * row['Hs']**2 * row['Tp'], axis=1)
Hrep1 = math.sqrt(sum(tableJul1['H2T'])/ sum(tableJul1['Tp']))
Hrep2 = math.sqrt(sum(tableJul2['H2T'])/ sum(tableJul2['Tp']))
Trep1 = tableJul1.Tp.mean()
Trep2 = tableJul2.Tp.mean()
thetarep1 = sum(tableJul1['thH2T'])/ sum(tableJul1['H2T'])
thetarep2 = sum(tableJul2['thH2T'])/ sum(tableJul2['H2T'])
print( '|  Jul  |  {:.2f} |  {:.2f} |   {:.1f}   | {:.1f} |'.format(Hrep1, Trep1, thetarep1, Pct1) )
print( '|  Jul  |  {:.2f} |  {:.2f} |   {:.1f}   | {:.1f} |'.format(Hrep2, Trep2, thetarep2, Pct2) )

Length of: subset 1 (549) and subset 2 (6499)
|  Jul  |  0.93 |  8.70 |   59.7   | 7.4 |
|  Jul  |  1.09 |  5.98 |   181.3   | 87.4 |


In [18]:
tableAug1 = tableAug[tableAug['Dir'].map(inGroup1)]
tableAug2 = tableAug[tableAug['Dir'].map(inGroup2)]
print( 'Length of: subset 1 ({}) and subset 2 ({})'.format(len(tableAug1), len(tableAug2)) )
Pct1 = len(tableAug1) / len(tableAug) * 100
Pct2 = len(tableAug2) / len(tableAug) * 100
tableAug1['H2T'] = tableAug1.apply(lambda row: row['Hs']**2 * row['Tp'], axis=1)
tableAug2['H2T'] = tableAug2.apply(lambda row: row['Hs']**2 * row['Tp'], axis=1)
tableAug1['thH2T'] = tableAug1.apply(lambda row: row['Dir'] * row['Hs']**2 * row['Tp'], axis=1)
tableAug2['thH2T'] = tableAug2.apply(lambda row: row['Dir'] * row['Hs']**2 * row['Tp'], axis=1)
Hrep1 = math.sqrt(sum(tableAug1['H2T'])/ sum(tableAug1['Tp']))
Hrep2 = math.sqrt(sum(tableAug2['H2T'])/ sum(tableAug2['Tp']))
Trep1 = tableAug1.Tp.mean()
Trep2 = tableAug2.Tp.mean()
thetarep1 = sum(tableAug1['thH2T'])/ sum(tableAug1['H2T'])
thetarep2 = sum(tableAug2['thH2T'])/ sum(tableAug2['H2T'])
print( '|  Aug  |  {:.2f} |  {:.2f} |   {:.1f}   | {:.1f} |'.format(Hrep1, Trep1, thetarep1, Pct1) )
print( '|  Aug  |  {:.2f} |  {:.2f} |   {:.1f}   | {:.1f} |'.format(Hrep2, Trep2, thetarep2, Pct2) )

Length of: subset 1 (783) and subset 2 (6100)
|  Aug  |  0.95 |  9.04 |   55.7   | 10.5 |
|  Aug  |  1.15 |  6.30 |   183.5   | 82.0 |


In [19]:
tableSep1 = tableSep[tableSep['Dir'].map(inGroup1)]
tableSep2 = tableSep[tableSep['Dir'].map(inGroup2)]
print( 'Length of: subset 1 ({}) and subset 2 ({})'.format(len(tableSep1), len(tableSep2)) )
Pct1 = len(tableSep1) / len(tableSep) * 100
Pct2 = len(tableSep2) / len(tableSep) * 100
tableSep1['H2T'] = tableSep1.apply(lambda row: row['Hs']**2 * row['Tp'], axis=1)
tableSep2['H2T'] = tableSep2.apply(lambda row: row['Hs']**2 * row['Tp'], axis=1)
tableSep1['thH2T'] = tableSep1.apply(lambda row: row['Dir'] * row['Hs']**2 * row['Tp'], axis=1)
tableSep2['thH2T'] = tableSep2.apply(lambda row: row['Dir'] * row['Hs']**2 * row['Tp'], axis=1)
Hrep1 = math.sqrt(sum(tableSep1['H2T'])/ sum(tableSep1['Tp']))
Hrep2 = math.sqrt(sum(tableSep2['H2T'])/ sum(tableSep2['Tp']))
Trep1 = tableSep1.Tp.mean()
Trep2 = tableSep2.Tp.mean()
thetarep1 = sum(tableSep1['thH2T'])/ sum(tableSep1['H2T'])
thetarep2 = sum(tableSep2['thH2T'])/ sum(tableSep2['H2T'])
print( '|  Sep  |  {:.2f} |  {:.2f} |   {:.1f}   | {:.1f} |'.format(Hrep1, Trep1, thetarep1, Pct1) )
print( '|  Sep  |  {:.2f} |  {:.2f} |   {:.1f}   | {:.1f} |'.format(Hrep2, Trep2, thetarep2, Pct2) )

Length of: subset 1 (3290) and subset 2 (3637)
|  Sep  |  1.17 |  8.20 |   63.8   | 45.7 |
|  Sep  |  1.05 |  6.06 |   178.5   | 50.5 |


In [20]:
tableOct1 = tableOct[tableOct['Dir'].map(inGroup1)]
tableOct2 = tableOct[tableOct['Dir'].map(inGroup2)]
print( 'Length of: subset 1 ({}) and subset 2 ({})'.format(len(tableOct1), len(tableOct2)) )
Pct1 = len(tableOct1) / len(tableOct) * 100
Pct2 = len(tableOct2) / len(tableOct) * 100
tableOct1['H2T'] = tableOct1.apply(lambda row: row['Hs']**2 * row['Tp'], axis=1)
tableOct2['H2T'] = tableOct2.apply(lambda row: row['Hs']**2 * row['Tp'], axis=1)
tableOct1['thH2T'] = tableOct1.apply(lambda row: row['Dir'] * row['Hs']**2 * row['Tp'], axis=1)
tableOct2['thH2T'] = tableOct2.apply(lambda row: row['Dir'] * row['Hs']**2 * row['Tp'], axis=1)
Hrep1 = math.sqrt(sum(tableOct1['H2T'])/ sum(tableOct1['Tp']))
Hrep2 = math.sqrt(sum(tableOct2['H2T'])/ sum(tableOct2['Tp']))
Trep1 = tableOct1.Tp.mean()
Trep2 = tableOct2.Tp.mean()
thetarep1 = sum(tableOct1['thH2T'])/ sum(tableOct1['H2T'])
thetarep2 = sum(tableOct2['thH2T'])/ sum(tableOct2['H2T'])
print( '|  Oct  |  {:.2f} |  {:.2f} |   {:.1f}   | {:.1f} |'.format(Hrep1, Trep1, thetarep1, Pct1) )
print( '|  Oct  |  {:.2f} |  {:.2f} |   {:.1f}   | {:.1f} |'.format(Hrep2, Trep2, thetarep2, Pct2) )

Length of: subset 1 (7061) and subset 2 (333)
|  Oct  |  1.89 |  8.46 |   47.8   | 94.9 |
|  Oct  |  1.37 |  6.00 |   166.3   | 4.5 |


In [21]:
tableNov1 = tableNov[tableNov['Dir'].map(inGroup1)]
tableNov2 = tableNov[tableNov['Dir'].map(inGroup2)]
print( 'Length of: subset 1 ({}) and subset 2 ({})'.format(len(tableNov1), len(tableNov2)) )
Pct1 = len(tableNov1) / len(tableNov) * 100
Pct2 = len(tableNov2) / len(tableNov) * 100
tableNov1['H2T'] = tableNov1.apply(lambda row: row['Hs']**2 * row['Tp'], axis=1)
tableNov2['H2T'] = tableNov2.apply(lambda row: row['Hs']**2 * row['Tp'], axis=1)
tableNov1['thH2T'] = tableNov1.apply(lambda row: row['Dir'] * row['Hs']**2 * row['Tp'], axis=1)
tableNov2['thH2T'] = tableNov2.apply(lambda row: row['Dir'] * row['Hs']**2 * row['Tp'], axis=1)
Hrep1 = math.sqrt(sum(tableNov1['H2T'])/ sum(tableNov1['Tp']))
Hrep2 = math.sqrt(sum(tableNov2['H2T'])/ sum(tableNov2['Tp']))
Trep1 = tableNov1.Tp.mean()
Trep2 = tableNov2.Tp.mean()
thetarep1 = sum(tableNov1['thH2T'])/ sum(tableNov1['H2T'])
thetarep2 = sum(tableNov2['thH2T'])/ sum(tableNov2['H2T'])
print( '|  Nov  |  {:.2f} |  {:.2f} |   {:.1f}   | {:.1f} |'.format(Hrep1, Trep1, thetarep1, Pct1) )
print( '|  Nov  |  {:.2f} |  {:.2f} |   {:.1f}   | {:.1f} |'.format(Hrep2, Trep2, thetarep2, Pct2) )

Length of: subset 1 (7151) and subset 2 (40)
|  Nov  |  2.57 |  8.79 |   49.8   | 99.3 |
|  Nov  |  2.76 |  7.11 |   123.0   | 0.6 |


In [22]:
tableDec1 = tableDec[tableDec['Dir'].map(inGroup1)]
tableDec2 = tableDec[tableDec['Dir'].map(inGroup2)]
print( 'Length of: subset 1 ({}) and subset 2 ({})'.format(len(tableDec1), len(tableDec2)) )
Pct1 = len(tableDec1) / len(tableDec) * 100
Pct2 = len(tableDec2) / len(tableDec) * 100
tableDec1['H2T'] = tableDec1.apply(lambda row: row['Hs']**2 * row['Tp'], axis=1)
tableDec2['H2T'] = tableDec2.apply(lambda row: row['Hs']**2 * row['Tp'], axis=1)
tableDec1['thH2T'] = tableDec1.apply(lambda row: row['Dir'] * row['Hs']**2 * row['Tp'], axis=1)
tableDec2['thH2T'] = tableDec2.apply(lambda row: row['Dir'] * row['Hs']**2 * row['Tp'], axis=1)
Hrep1 = math.sqrt(sum(tableDec1['H2T'])/ sum(tableDec1['Tp']))
Hrep2 = math.sqrt(sum(tableDec2['H2T'])/ sum(tableDec2['Tp']))
Trep1 = tableDec1.Tp.mean()
Trep2 = tableDec2.Tp.mean()
thetarep1 = sum(tableDec1['thH2T'])/ sum(tableDec1['H2T'])
thetarep2 = sum(tableDec2['thH2T'])/ sum(tableDec2['H2T'])
print( '|  Dec  |  {:.2f} |  {:.2f} |   {:.1f}   | {:.1f} |'.format(Hrep1, Trep1, thetarep1, Pct1) )
print( '|  Dec  |  {:.2f} |  {:.2f} |   {:.1f}   | {:.1f} |'.format(Hrep2, Trep2, thetarep2, Pct2) )

Length of: subset 1 (7426) and subset 2 (9)
|  Dec  |  3.01 |  9.08 |   51.3   | 99.8 |
|  Dec  |  1.40 |  6.83 |   105.8   | 0.1 |


## Wave conditions

Gathering the above print-outs, we can create a markdown table including two conditions per month:

| Month | Hsrep | Tprep | thetarep |  %   |
|:-----:|:-----:|:-----:|:--------:|:----:|
|  Jan  |  2.42 |  8.46 |   51.7   | 99.7 |
|  Jan  |  1.39 |  4.61 |   120.8  |  0.2 |
|  Feb  |  2.04 |  8.14 |   52.2   | 94.4 |
|  Feb  |  0.66 |  4.58 |   122.9  |  5.3 |
|  Mar  |  1.66 |  7.74 |   51.7   | 87.3 |
|  Mar  |  0.75 |  4.19 |   141.8  | 12.3 |
|  Apr  |  1.18 |  7.38 |   52.5   | 66.6 |
|  Apr  |  0.85 |  4.21 |   164.3  | 32.8 |
|  May  |  1.02 |  7.16 |   53.5   | 38.7 |
|  May  |  0.92 |  4.80 |   174.2  | 59.6 |
|  Jun  |  0.84 |  6.69 |   56.1   |  9.5 |
|  Jun  |  1.04 |  5.64 |   179.5  | 84.8 |
|  Jul  |  0.93 |  8.70 |   59.7   |  7.4 |
|  Jul  |  1.09 |  5.98 |   181.3  | 87.4 |
|  Aug  |  0.95 |  9.04 |   55.7   | 10.5 |
|  Aug  |  1.15 |  6.30 |   183.5  | 82.0 |
|  Sep  |  1.17 |  8.20 |   63.8   | 45.7 |
|  Sep  |  1.05 |  6.06 |   178.5  | 50.5 |
|  Oct  |  1.89 |  8.46 |   47.8   | 94.9 |
|  Oct  |  1.37 |  6.00 |   166.3  |  4.5 |
|  Nov  |  2.57 |  8.79 |   49.8   | 99.3 |
|  Nov  |  2.76 |  7.11 |   123.0  |  0.6 |
|  Dec  |  3.01 |  9.08 |   51.3   | 99.8 |
|  Dec  |  1.40 |  6.83 |   105.8  |  0.1 |

Filtering the about results, keeping only either predominant conditions with > 90% occurence, or high wave conditions. These 21 conditions are used for analysis, as described in the paper.

| Month | Hsrep | Tprep | thetarep |  %   |
|:-----:|:-----:|:-----:|:--------:|:----:|
|  Jan  |  2.42 |  8.46 |   51.7   | 99.7 |
|  Feb  |  2.04 |  8.14 |   52.2   | 94.4 |
|  Mar  |  1.66 |  7.74 |   51.7   | 87.3 |
|  Mar  |  0.75 |  4.19 |   141.8  | 12.3 |
|  Apr  |  1.18 |  7.38 |   52.5   | 66.6 |
|  Apr  |  0.85 |  4.21 |   164.3  | 32.8 |
|  May  |  1.02 |  7.16 |   53.5   | 38.7 |
|  May  |  0.92 |  4.80 |   174.2  | 59.6 |
|  Jun  |  0.84 |  6.69 |   56.1   |  9.5 |
|  Jun  |  1.04 |  5.64 |   179.5  | 84.8 |
|  Jul  |  0.93 |  8.70 |   59.7   |  7.4 |
|  Jul  |  1.09 |  5.98 |   181.3  | 87.4 |
|  Aug  |  0.95 |  9.04 |   55.7   | 10.5 |
|  Aug  |  1.15 |  6.30 |   183.5  | 82.0 |
|  Sep  |  1.17 |  8.20 |   63.8   | 45.7 |
|  Sep  |  1.05 |  6.06 |   178.5  | 50.5 |
|  Oct  |  1.89 |  8.46 |   47.8   | 94.9 |
|  Nov  |  2.57 |  8.79 |   49.8   | 99.3 |
|  Dec  |  3.01 |  9.08 |   51.3   | 99.8 |